In [1]:
from google.colab.drive import mount
mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
drive_path = 'drive/My Drive/'

In [0]:
!sudo apt-get install unar
!pip install pydicom

In [0]:
!cp 'drive/My Drive/INbreast.rar' INbreast.rar
!unar 'INbreast.rar'

In [0]:
import os
import pydicom 
import numpy as np 
import matplotlib.pyplot as plt
import cv2
from random import shuffle
import glob
from numpy.linalg import norm 
import pandas as pd
import csv

In [0]:
images = glob.glob('INbreast/AllDICOMs/*.dcm')

In [0]:
csv_path = 'INbreast/INbreast.xls'
df = pd.read_excel(csv_path)

filenames = df['File Name'].tolist()
labels   = df['Bi-Rads'].tolist()

data = dict(zip(filenames, labels))

In [0]:
IMG_SIZE = 224

In [0]:
import skimage.feature as ft

def get_id_from_mask(path):
  return path[:-9]

def get_image_from_id(id):
  img_paths = glob.glob('INbreast/AllDICOMs/*.dcm')
  for img_path in img_paths:
    if id in img_path:
      return img_path
    
def get_array_from_dicom(path):
  img = pydicom.dcmread(path).pixel_array
  return img

def get_mammo_from_mask(msk_path):
  id = get_id_from_mask(msk_path)
  img_path  = get_image_from_id(id)
  img = get_array_from_dicom(img_path)
  return img_path, img

def map_label(fname):
  for f in data.keys():
    l = int(f) 
    if str(l) in fname:
      label = data[f]
      break
  if label in ['4a', '4b', '4c']:
    label = 4
  
  return int(label) -2

def preprocess(img):
  img = cv2.resize(img, dsize = (IMG_SIZE, IMG_SIZE))
  #img = np.stack((img,)*3, axis=-1)
  img = np.expand_dims(img, -1)
  return img.astype('float32')

def resize_and_crop(img):
  result = np.zeros((1685, 1685))

  if img.max() > 0:
    mask = img == 0
    rows = np.flatnonzero((~mask).sum(axis=1))
    cols = np.flatnonzero((~mask).sum(axis=0))

    x0 = rows.min()
    x1 = rows.max()+1
    y0 = cols.min()
    y1 = cols.max()+1

    v = img
    v = v[x0:x1, y0:y1]
    result[:v.shape[0],:v.shape[1]] = v
  return preprocess(result)

In [0]:
mass_msk_base = 'INbreast/extras/MassSegmentationMasks/'
mc_msk_base = 'INbreast/extras/CalcificationSegmentationMasks/'

msk_files = list(set(os.listdir(mass_msk_base)) | set(os.listdir(mc_msk_base)))

features = []
labels = []

added_so_far = []
for msk_file  in msk_files:
  #get full mammogram 
  
  try:
    img_path, img = get_mammo_from_mask(msk_file)
    added_so_far.append(img_path)
  except:
    continue

  #check of MC view exists
  if msk_file in os.listdir(mc_msk_base):
    mc_msk = cv2.imread(mc_msk_base+msk_file)[...,0]
  else:
    mc_msk = np.zeros_like(img)

  #check if 
  if msk_file in os.listdir(mass_msk_base):
    mass_msk = cv2.imread(mass_msk_base+msk_file)[...,0]
  else:
    mass_msk = np.zeros_like(img) 


  #get the mc mask 
  label =  map_label(img_path)
  img = img/img.max()
  mass = mass_msk/255
  mc   = mc_msk/255

  processed_mass = resize_and_crop(img*mass)
  processed_mc = preprocess(img*mc)
  processed_img = preprocess(img)
  
  #print(hog_feature.shape, lbp_feature.shape)
  features.append(np.concatenate([processed_img, processed_mass, processed_mc], axis = -1))
  labels.append(label)

In [107]:
np.min(labels)

0

In [0]:
import keras
from keras.callbacks import Callback
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten,\
 Conv2D, MaxPooling2D, Input, Concatenate, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
import numpy as np
import keras.backend as K
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score , f1_score, roc_auc_score       
from sklearn.model_selection import StratifiedKFold, KFold
from keras.optimizers import Adam

In [0]:
def conv_block(fs, x):
  conv  = Conv2D(fs, (3, 3), padding = 'same', activation = 'relu')(x)
  pool  = MaxPooling2D(pool_size = (2, 2))(conv)
  return pool

def res_block(fs, x):
  conv  = Conv2D(fs, (3, 3), padding = 'same', activation = 'relu')(x)
  pool  = MaxPooling2D(pool_size = (2, 2))(Concatenate(axis = -1)([x, conv]))
  return pool

def to_labels(pred):
  return np.argmax(pred, axis = 1)

def model1():
  inp = Input(shape = (224, 224, 3))

  block1 = conv_block(32, inp)
  block2 = conv_block(64, block1)
  block3 = conv_block(64, block2)
  block4 = conv_block(128, block3)
  block5 = conv_block(128, block4)

  flatten = Flatten()(block5)

  out = Dense(128, activation = 'relu')(flatten)
  out = Dense(5, activation = 'softmax')(out)

  adam = Adam(lr = 0.0001)
  model = keras.models.Model(inputs = inp, outputs = out)
  model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
  return model

def resnet():
  inp = Input(shape = (224, 224, 3))

  block1 = res_block(32, inp)
  block2 = res_block(64, block1)
  block3 = res_block(64, block2)
  block4 = res_block(128, block3)
  block5 = res_block(128, block4)
  flatten = Flatten()(block5)

  out = Dense(128, activation = 'relu')(flatten)
  out = Dense(5, activation = 'softmax')(out)

  model = keras.models.Model(inputs = inp, outputs = out)
  model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
  return model

def fine_tune_vgg():
  #download and load the pretrained model
  vgg = keras.applications.VGG16(input_shape = (IMG_SIZE, IMG_SIZE, 3), weights ='imagenet', include_top = False)

  #freeze all the layers as a fixed feature extractor
  for layer in vgg.layers:
    layer.trainable = False

  flatten = GlobalAveragePooling2D()(vgg.output)

  #first branch for classification
  branch1 = Dense(1024, activation = 'relu')(flatten)
  out = Dense(5, activation = 'softmax')(branch1)

  #create a model with one input and two outputs 
  model = keras.models.Model(inputs = vgg.input , outputs = out)
  model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

def fine_tune_resnet():
  #download and load the pretrained model
  vgg = keras.applications.ResNet50(input_shape = (IMG_SIZE, IMG_SIZE, 3), weights ='imagenet', include_top = False)

  #freeze all the layers as a fixed feature extractor
  for layer in vgg.layers:
    layer.trainable = False

  flatten = GlobalAveragePooling2D()(vgg.output)

  #first branch for classification
  branch1 = Dense(1024, activation = 'relu')(flatten)
  out = Dense(6, activation = 'softmax')(branch1)

  #create a model with one input and two outputs 
  model = keras.models.Model(inputs = vgg.input , outputs = out)
  model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

def fine_tune_incept():
  #download and load the pretrained model
  vgg = keras.applications.InceptionResNetV2(input_shape = (IMG_SIZE, IMG_SIZE, 3), weights ='imagenet', include_top = False)

  #freeze all the layers as a fixed feature extractor
  for layer in vgg.layers:
    layer.trainable = False

  flatten = GlobalAveragePooling2D()(vgg.output)

  #first branch for classification
  branch1 = Dense(1024, activation = 'relu')(flatten)
  out = Dense(6, activation = 'softmax')(branch1)

  #create a model with one input and two outputs 
  model = keras.models.Model(inputs = vgg.input , outputs = out)
  model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

In [0]:
kfold = KFold(n_splits=10, shuffle=True)
X = np.array(features)
Y = np.array(labels)

In [130]:
cvscores = {'acc':[], 'f_1':[], 'prc':[], 'rec':[], 'roc':[0]}

for i, (train, test) in enumerate(kfold.split(X, Y)):
  #model = model1()
  #model = resnet()
  model = fine_tune_vgg()
  #model = fine_tune_incept()
  model.fit(X[train], Y[train], batch_size = 64, epochs=80, verbose = 0) #resnet model 
  #model.fit(X[train], Y[train], batch_size = 16, epochs=15, verbose = 0) #model1  
  # evaluate the model
  logits = model.predict(X[test])
  pred = to_labels(logits)
  s1 = accuracy_score(Y[test], pred)
  s2 = f1_score(Y[test], pred, average='weighted')
  s3 = precision_score(Y[test], pred, average='weighted')
  s4 = recall_score(Y[test], pred, average='weighted')
  #s5 = roc_auc_score(Y[test], logits)
  
  print(f'{i}: acc:{s1:.2f}  f_1:{s2:.2f}  prc:{s3:.2f}  rec:{s4:.2f}')

  cvscores['acc'].append(s1)
  cvscores['f_1'].append(s2)
  cvscores['prc'].append(s3)
  cvscores['rec'].append(s4)
  #cvscores['roc'].append(s5)

print('======================================')
avgs = [np.mean(cvscores['acc']), np.mean(cvscores['f_1']), np.mean(cvscores['prc']), np.mean(cvscores['rec']), np.mean(cvscores['roc'])]
print(f'acc:{avgs[0]:.2f}  f_1:{avgs[1]:.2f}  prc:{avgs[2]:.2f}  rec:{avgs[3]:.2f} roc:{avgs[4]:.2f}')

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0: acc:0.75  f_1:0.71  prc:0.67  rec:0.75
1: acc:0.75  f_1:0.72  prc:0.81  rec:0.75
2: acc:0.60  f_1:0.57  prc:0.54  rec:0.60
3: acc:0.75  f_1:0.72  prc:0.71  rec:0.75
4: acc:0.65  f_1:0.60  prc:0.59  rec:0.65
5: acc:0.65  f_1:0.64  prc:0.67  rec:0.65
6: acc:0.70  f_1:0.64  prc:0.61  rec:0.70
7: acc:0.60  f_1:0.51  prc:0.48  rec:0.60
8: acc:0.85  f_1:0.81  prc:0.77  rec:0.85
9: acc:0.63  f_1:0.58  prc:0.55  rec:0.63
acc:0.69  f_1:0.65  prc:0.64  rec:0.69 roc:0.00
